In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Load and preprocess the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
X_test = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0

# Apply PCA to reduce dimensionality to 150
pca = PCA(n_components=150)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# One-hot encode labels
y_train_encoded = np.eye(10)[y_train]
y_test_encoded = np.eye(10)[y_test]

class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.001):
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
        self.bias_hidden = np.zeros(hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
        self.bias_output = np.zeros(output_size)
        self.learning_rate = learning_rate

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        # Forward pass
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.relu(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        return self.softmax(self.final_input)

    def backward(self, X, y, output):
        # Calculate gradients
        error = output - y
        d_weights_hidden_output = np.dot(self.hidden_output.T, error)
        d_bias_output = np.sum(error, axis=0)

        hidden_error = np.dot(error, self.weights_hidden_output.T) * self.relu_derivative(self.hidden_input)
        d_weights_input_hidden = np.dot(X.T, hidden_error)
        d_bias_hidden = np.sum(hidden_error, axis=0)

        # Update weights and biases
        self.weights_hidden_output -= self.learning_rate * d_weights_hidden_output
        self.bias_output -= self.learning_rate * d_bias_output
        self.weights_input_hidden -= self.learning_rate * d_weights_input_hidden
        self.bias_hidden -= self.learning_rate * d_bias_hidden

    def compute_loss(self, y_true, y_pred):
        # Cross-entropy loss
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true.argmax(axis=1)] + 1e-8)
        return np.sum(log_likelihood) / m

    def train(self, X, y, epochs, batch_size=64):
        train_losses, train_accuracies = [], []

        for epoch in range(epochs):
            indices = np.random.permutation(X.shape[0])
            X, y = X[indices], y[indices]

            for i in range(0, X.shape[0], batch_size):
                batch_X = X[i:i + batch_size]
                batch_y = y[i:i + batch_size]

                output = self.forward(batch_X)
                self.backward(batch_X, batch_y, output)

            # Calculate loss and accuracy after each epoch
            train_output = self.forward(X)
            train_loss = self.compute_loss(y, train_output)
            train_accuracy = np.mean(np.argmax(train_output, axis=1) == np.argmax(y, axis=1))
            train_losses.append(train_loss)
            train_accuracies.append(train_accuracy)
            
            print(f"Epoch {epoch + 1}/{epochs} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

        # Plot the loss and accuracy
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(range(epochs), train_losses, label='Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(range(epochs), train_accuracies, label='Training Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    def evaluate(self, X, y):
        # Evaluation on the test set
        test_output = self.forward(X)
        test_loss = self.compute_loss(y, test_output)
        test_accuracy = np.mean(np.argmax(test_output, axis=1) == np.argmax(y, axis=1))
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Create and train the neural network
model = SimpleNN(input_size=150, hidden_size=256, output_size=10, learning_rate=0.001)
model.train(X_train, y_train_encoded, epochs=25, batch_size=64)

# Evaluate on test data
model.evaluate(X_test, y_test_encoded)